## Preprocessing

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

train = pd.read_csv('/home/mahdi/My/Project/zExcel/Kaggle/Titanic/train.csv')
test  = pd.read_csv('/home/mahdi/My/Project/zExcel/Kaggle/Titanic/test.csv')       
DATA = pd.concat([train, test], axis=0, ignore_index=True)

def preprocessing():
    global data

    data = pd.concat([train, test], axis=0, ignore_index=True)

    del data['Name'],data['Cabin'],data['Ticket'],data['PassengerId']

    data["Age"] = data["Age"].fillna(data.groupby("Sex")["Age"].transform("median"))

    data['Embarked'] = data['Embarked'].fillna('S')
    data = pd.get_dummies(data ,columns=['Embarked'],dtype=np.float64)

    data['Fare'] = data['Fare'].fillna(data.groupby("Pclass")["Fare"].transform("mean"))

    data['Sex'].replace({'male':0 ,'female':1} ,inplace=True)



def Outliers():                                                            
    flt = (data['Fare'] >= 400) & (data['Survived'].notnull())
    data.drop(data[flt].index, inplace=True)


preprocessing()

Outliers()

## Normalize & Split

In [9]:
Train = data[data['Survived'].notnull()]
Test = data[data['Survived'].isnull()]

train = MinMaxScaler().fit_transform(Train)
test = MinMaxScaler().fit_transform(Test)

X = train[:,1:]
y = train[: ,0]

TEST = test[: ,1:]

x_train , x_test , y_train , y_test = train_test_split(X, y, test_size=0.1 , random_state=42 )
x_train.shape ,y_train.shape

((799, 9), (799,))

## Modeling

In [10]:
tree = DecisionTreeClassifier(max_depth=4, random_state=42, ccp_alpha=0.001)

model = BaggingClassifier(base_estimator=tree, n_estimators=70, oob_score=True,   
                        max_samples=0.75,                                          
                        n_jobs=12 ,random_state=42)                 

model.fit(x_train, y_train)

Predict = model.predict(TEST)
Kaggle = pd.DataFrame(DATA[DATA['Survived'].isnull()]['PassengerId'])
Kaggle ['Survived'] = Predict
Kaggle ['Survived'] = Kaggle ['Survived'].astype(int)

model.score(x_test, y_test)

0.8876404494382022